In [1]:
"""
Use of same experimental setup as for our Probabilistic Suffix Prediction.

Reimplementation for comparison: 
- Paper: Camargo, Manuel, Marlon Dumas, and Oscar González-Rojas. "Learning accurate LSTM models of business processes." International Conference on Business Process Management. Cham: Springer International Publishing, 2019.
- Github (code) from: https://github.com/AdaptiveBProcess/GenerativeLSTM/tree/master/
"""

'\nUse of same experimental setup as for our Probabilistic Suffix Prediction.\n\nReimplementation for comparison: \n- Paper: Camargo, Manuel, Marlon Dumas, and Oscar González-Rojas. "Learning accurate LSTM models of business processes." International Conference on Business Process Management. Cham: Springer International Publishing, 2019.\n- Github (code) from: https://github.com/AdaptiveBProcess/GenerativeLSTM/tree/master/\n'

# Imports

In [2]:
import importlib
import sys
import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

# Data

### Load Data Files

In [3]:
# Path to your pickle file (saved with torch.save)
file_path_train = '../../../../../../encoded_data/compare_camargo/Sepsis_all_5_train.pkl'
# Load the dataset using torch.load
sepsis_train_dataset = torch.load(file_path_train, weights_only=False)
# Check the type of the loaded dataset
print(type(sepsis_train_dataset))

# Path to your pickle file (saved with torch.save)
file_path_val = '../../../../../../encoded_data/compare_camargo/Sepsis_all_5_val.pkl'
# Load the dataset using torch.load
sepsis_val_dataset = torch.load(file_path_val, weights_only=False)
# Check the type of the loaded dataset
print(type(sepsis_val_dataset))


<class 'event_log_loader.new_event_log_loader.EventLogDataset'>
<class 'event_log_loader.new_event_log_loader.EventLogDataset'>


### Train Data Insights

In [ ]:
# Sepsis Dataset Categories, Features:
sepsis_all_categories = sepsis_train_dataset.all_categories

sepsis_all_categories_cat = sepsis_all_categories[0]
print(sepsis_all_categories_cat)

sepsis_all_categories_num = sepsis_all_categories[1]
print(sepsis_all_categories_num)

for i, cat in enumerate(sepsis_all_categories_cat):
     print(f"sepsis(5) Categorical feature: {cat[0]}, Index position in categorical data list: {i}")
     print(f"sepsis (5) Total Amount of Category labels: {cat[1]}")

print('\n')    

for i, num in enumerate(sepsis_all_categories_num):
     print(f"sepsis (5) Numerical feature: {num[0]}, Index position in categorical data list: {i}")
     print(f"sepsis (5) Amount Numerical: {num[1]}")
     
# Get concept_name id:
concept_name = 'concept:name'
concept_name_id = [i for i, cat in enumerate(sepsis_all_categories[0]) if cat[0] == concept_name][0]
print("ID concet name in cat list: ", concept_name_id)

# Output size
concept_name = 'concept:name'
concept_name_size = [cat[1] for _, cat in enumerate(sepsis_all_categories[0]) if cat[0] == concept_name][0]
print("ID concet name in cat list: ", concept_name_size)

# Id of EOS token in activity
eos_value = 'EOS'
eos_id = [v for k, v in sepsis_all_categories[0][concept_name_id][2].items() if k == eos_value][0]
# Get EOS id of concept name list:
print("ID EOS in concept name tensor: ", eos_id)


[('concept:name', 18, {'Admission IC': 1, 'Admission NC': 2, 'CRP': 3, 'EOS': 4, 'ER Registration': 5, 'ER Sepsis Triage': 6, 'ER Triage': 7, 'IV Antibiotics': 8, 'IV Liquid': 9, 'LacticAcid': 10, 'Leucocytes': 11, 'Release A': 12, 'Release B': 13, 'Release C': 14, 'Release D': 15, 'Release E': 16, 'Return ER': 17}), ('org:group', 27, {'?': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'EOS': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26})]
[('case_elapsed_time', 1, {})]
sepsis(5) Categorical feature: concept:name, Index position in categorical data list: 0
sepsis (5) Total Amount of Category labels: 18
sepsis(5) Categorical feature: org:group, Index position in categorical data list: 1
sepsis (5) Total Amount of Category labels: 27


sepsis (5) Numerical feature: case_elapsed_time, Index position in categorical data list: 0
sepsis (5) Amount Numerical: 1
ID concet 

### Input Features for Encoder and Decoder

In [5]:
# Create lists with name of Model features (input)
model_feat_cat = []
model_feat_num = []
for cat in sepsis_all_categories_cat:
    model_feat_cat.append(cat[0])
for num in sepsis_all_categories_num:
    model_feat_num.append(num[0])
model_feat = [model_feat_cat, model_feat_num]
print("Input features encoder: ", model_feat)


Input features encoder:  [['concept:name', 'org:group'], ['case_elapsed_time']]


# Model

In [6]:
import joinLSTM.model
importlib.reload(joinLSTM.model)
from joinLSTM.model import FullShared_Join_LSTM

"""
Specific model parameters from paper: 
"""

# Size hidden layer
hidden_size= 50

# Number of LSTM cells
num_layers = 1

# STANDARD: One numerical output to predict
input_size = 1

# Hans Weytjens LSTM model
model = FullShared_Join_LSTM(data_set_categories=sepsis_all_categories,
                             hidden_size=hidden_size,
                             num_layers=num_layers,
                             model_feat=model_feat,
                             input_size=input_size,
                             output_size_act=concept_name_size)

Data set categories:  ([('concept:name', 18, {'Admission IC': 1, 'Admission NC': 2, 'CRP': 3, 'EOS': 4, 'ER Registration': 5, 'ER Sepsis Triage': 6, 'ER Triage': 7, 'IV Antibiotics': 8, 'IV Liquid': 9, 'LacticAcid': 10, 'Leucocytes': 11, 'Release A': 12, 'Release B': 13, 'Release C': 14, 'Release D': 15, 'Release E': 16, 'Return ER': 17}), ('org:group', 27, {'?': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'EOS': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26})], [('case_elapsed_time', 1, {})])
Model input features:  [['concept:name', 'org:group'], ['case_elapsed_time']]


Embeddings:  ModuleList(
  (0): Embedding(18, 8)
  (1): Embedding(27, 10)
)
Total embedding feature size:  18
Input feature size:  19
Cells hidden size:  50
Number of LSTM layer:  1


/home/LordKunkler/PrimePPM/.venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


# Training Configuration

In [7]:
import training.train
importlib.reload(training.train)
from training.train import Training

from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(comment="Full_sepsis_camargo_act")

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
Parameter of Probabilistic Suffix Prediction experimental design, to ensure fair comparison:
"""

# Start learning rate
learning_rate = 1e-5

# Optimizer and Scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, min_lr=1e-10)

# Epochs
num_epochs = 100

# Batch of model input
batch_size = 128

# shuffle data
shuffle = True

optimize_values = {"optimizer":optimizer,
                   "scheduler": scheduler,
                   "epochs":num_epochs,
                   "mini_batches":batch_size,
                   "shuffle": shuffle}

number_tasks = len(model_feat)

trainer = Training(model=model,
                   device=device,
                   data_train=sepsis_train_dataset,
                   data_val=sepsis_val_dataset,
                   optimize_values=optimize_values,
                   concept_name_id=concept_name_id,
                   eos_id=eos_id,
                   writer=writer,
                   save_model_n_th_epoch=1,
                   saving_path="Sepsis_camargo_act_1_suffix_length2.pkl")

# Train the model:
trainer.train()

Device:  cuda
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0
)
Scheduler:  <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7fe98e47c440>
Epochs:  100
Mini baches:  128
Shuffle batched dataset:  True


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Learning Rate: 1e-05
Training: Avg Attenuated Training Loss: 2.8861


Validation: Avg Validation Loss: 2.8841
Validation Loss for Scheduler: 2.8841
saving model


Epoch [2/100], Learning Rate: 1e-05
Training: Avg Attenuated Training Loss: 2.8820


...

Validation: Avg Validation Loss: 2.5344
Validation Loss for Scheduler: 2.5344
saving model


Epoch [99/100], Learning Rate: 1e-05
Training: Avg Attenuated Training Loss: 2.5329


Validation: Avg Validation Loss: 2.5337
Validation Loss for Scheduler: 2.5337
saving model


Epoch [100/100], Learning Rate: 1e-05
Training: Avg Attenuated Training Loss: 2.5325


Validation: Avg Validation Loss: 2.5345
Validation Loss for Scheduler: 2.5345
saving model
Training complete.
Model saved to path: Sepsis_camargo_act_1_suffix_length2.pkl
